In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/loan-approval-prediction-dataset/loan_approval_dataset.csv


In [2]:
loan_data = pd.read_csv("/kaggle/input/loan-approval-prediction-dataset/loan_approval_dataset.csv")
loan_data.head(10)

,loan_id,no_of_dependents,education,self_employed,income_annum,loan_amount,loan_term,cibil_score,residential_assets_value,commercial_assets_value,luxury_assets_value,bank_asset_value,loan_status
0,1,2,Graduate,No,9600000,29900000,12,778,2400000,17600000,22700000,8000000,Approved
1,2,0,Not Graduate,Yes,4100000,12200000,8,417,2700000,2200000,8800000,3300000,Rejected
2,3,3,Graduate,No,9100000,29700000,20,506,7100000,4500000,33300000,12800000,Rejected
3,4,3,Graduate,No,8200000,30700000,8,467,18200000,3300000,23300000,7900000,Rejected
4,5,5,Not Graduate,Yes,9800000,24200000,20,382,12400000,8200000,29400000,5000000,Rejected
5,6,0,Graduate,Yes,4800000,13500000,10,319,6800000,8300000,13700000,5100000,Rejected
6,7,5,Graduate,No,8700000,33000000,4,678,22500000,14800000,29200000,4300000,Approved
7,8,2,Graduate,Yes,5700000,15000000,20,382,13200000,5700000,11800000,6000000,Rejected
8,9,0,Graduate,Yes,800000,2200000,20,782,1300000,800000,2800000,600000,Approved
9,10,5,Not Graduate,No,1100000,4300000,10,388,3200000,1400000,3300000,1600000,Rejected


In [3]:
df=loan_data

In [4]:
df.isnull().sum()

loan_id                      0
 no_of_dependents            0
 education                   0
 self_employed               0
 income_annum                0
 loan_amount                 0
 loan_term                   0
 cibil_score                 0
 residential_assets_value    0
 commercial_assets_value     0
 luxury_assets_value         0
 bank_asset_value            0
 loan_status                 0
dtype: int64

In [5]:
label_encoder = preprocessing.LabelEncoder()
loan_data.columns

NameError: name 'preprocessing' is not defined

In [ ]:
label_encoder_y = preprocessing.LabelEncoder() 

In [ ]:
loan_data[' education'] = label_encoder.fit_transform(loan_data[' education'])
loan_data[' self_employed'] = label_encoder.fit_transform(loan_data[' self_employed'])
loan_data[' loan_status'] = label_encoder_y.fit_transform(loan_data[' loan_status'])
print(loan_data[[' education', ' self_employed',' loan_status']])

In [ ]:
loan_data

In [ ]:
selected_features = [' education', ' self_employed', ' loan_status', ' no_of_dependents',  ' income_annum', ' loan_amount',' loan_term',' cibil_score',' residential_assets_value', ' commercial_assets_value',
       ' luxury_assets_value', ' bank_asset_value']
selected_features

In [ ]:
X = df[selected_features]  # Bağımsız Değerler
y = df[' loan_status']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
mms = MinMaxScaler()

X_train = mms.fit_transform(X_train)
X_test = mms.transform(X_test)


In [ ]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
n_input = X_train.shape[1]
n_output = len(np.unique(y_train))

print('Input Neuron:', n_input)
print('Output Neuron:', n_output)

In [ ]:
class LVQ(object):
    def __init__(self, sizeInput, sizeOutput, max_epoch, learning_rate, fungsi_pembelajaran):
        self.sizeInput = sizeInput
        self.sizeOutput = sizeOutput
        self.max_epoch = max_epoch
        self.alpha = learning_rate
        self.fungsi_pembelajaran = fungsi_pembelajaran
        self.weight = np.zeros((sizeOutput, sizeInput))

    def getWeight(self):
        return self.weight

    def train(self, train_data, train_target):
        weight_label, label_index = np.unique(train_target, return_index=True)

        # Inisialisasi bobot
        self.weight = train_data[label_index].astype(np.float)

        # Hapus data yang digunakan untuk inisialisasi bobot
        train_data = np.delete(train_data, label_index, axis=0)
        train_target = np.delete(train_target, label_index, axis=0)

        epoch = 0
        iterasi = 0
        
        while epoch <= self.max_epoch:
            epoch += 1
            for data, target in zip(train_data, train_target):
                iterasi += 1
                distance = np.sqrt(np.sum((data - self.weight) ** 2, axis=1))
                idx_min = np.argmin(distance)

                if target == weight_label[idx_min]:
                    self.weight[idx_min] = self.weight[idx_min] + self.alpha * (data - self.weight[idx_min])
                else:
                    self.weight[idx_min] = self.weight[idx_min] - self.alpha * (data - self.weight[idx_min])

            self.alpha = self.alpha - (self.fungsi_pembelajaran * self.alpha)

        weight_class = (self.weight, weight_label)
        return weight_class

    def test(self, test_data, weight_class):
        weight, label = weight_class
        output = []
        for data in test_data:
            distance = np.sqrt(np.sum((data - self.weight) ** 2, axis=1))
            idx_min = np.argmin(distance)
            output.append(label[idx_min])

        return output


In [ ]:
class LVQ(object):
    def __init__(self, sizeInput, sizeOutput, max_epoch, learning_rate, fungsi_pembelajaran):
        self.sizeInput = sizeInput
        self.sizeOutput = sizeOutput
        self.max_epoch = max_epoch
        self.alpha = learning_rate
        self.fungsi_pembelajaran = fungsi_pembelajaran
        self.weight = np.zeros((sizeOutput, sizeInput))

    def getWeight(self):
        return self.weight

    def train(self, train_data, train_target):
        weight_label, label_index = np.unique(train_target, return_index=True)

        # Initialize weights
        self.weight = train_data[label_index].astype(float)

        # Remove data used for weight initialization
        train_data = np.delete(train_data, label_index, axis=0)
        train_target = np.delete(train_target, label_index, axis=0)

        epoch = 0
        
        while epoch <= self.max_epoch:
            epoch += 1
            for data, target in zip(train_data, train_target):
                distance = np.sqrt(np.sum((data - self.weight) ** 2, axis=1))
                idx_min = np.argmin(distance)

                if target == weight_label[idx_min]:
                    self.weight[idx_min] = self.weight[idx_min] + self.alpha * (data - self.weight[idx_min])
                else:
                    self.weight[idx_min] = self.weight[idx_min] - self.alpha * (data - self.weight[idx_min])

            self.alpha = self.alpha - (self.fungsi_pembelajaran * self.alpha)

        weight_class = (self.weight, weight_label)
        return weight_class

    def test(self, test_data, weight_class):
        weight, label = weight_class
        output = []
        for data in test_data:
            distance = np.sqrt(np.sum((data - self.weight) ** 2, axis=1))
            idx_min = np.argmin(distance)
            output.append(label[idx_min])

        return output


In [ ]:
lvq = LVQ(sizeInput=n_input, sizeOutput=n_output, max_epoch=3, learning_rate=0.3, fungsi_pembelajaran=0.1)
bobot_dan_label = lvq.train(X_train, y_train)
bobot = lvq.getWeight()

print('Bobot: ', bobot)
print('Ukuran Bobot:', bobot.shape)

In [ ]:
y_pred = lvq.test(X_test, bobot_dan_label)
print('Accuracy:', accuracy_score(y_test, y_pred))

In [ ]:
print(confusion_matrix(y_pred, y_test))

In [ ]:
print(classification_report(y_pred, y_test))

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# Test verileri üzerinde tahmin yapma
y_pred = lvq.test(X_test, bobot_dan_label)
# Tahmin edilen sınıfları geri dönüştürme (one-hot encoding'den geri dönüşüm)
y_pred_classes = np.array(y_pred)
# Gerçek sınıfları geri dönüştürme (one-hot encoding'den geri dönüşüm)
y_true = y_test

# Confusion matrix'i hesaplama
conf_matrix = confusion_matrix(y_true, y_pred_classes)

# Renkli görselleştirme
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
plt.title("Confusion Matrix")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()